# Open Stack Swift storage of the project

In [ ]:
# default_exp datasets

## Module Installation

In [ ]:
#!pip install --user python-swiftclient python-keystoneclient --upgrade
#!pip install nbdev

## Class Datasets
Using a config file for credentials

In [ ]:
#export
import swiftclient
import json
import glob
import os
#from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
import pandas as pd

class Datasets:
    """
    Utility class to access the Open Stack Swift storage of the project.
    """
    config = None # Dict configuration
    conn = None # swiftclient.Connection object
    container_name = 'oco2'
    
    def __init__(self, config_file):
        """
        Constructor
        :param config_file: str, Path to config file
        :return:
        """
        # Load config
        with open(config_file) as json_data_file:
            self.config = json.load(json_data_file)
        self.conn = self.swift_con()

    def swift_con(self, config=None):
        """
        Connect to Open Stack Swift
        :param config: dict, Config dictionary.
        :return: swiftclient.Connection
        """
        if config is None:
            config = self.config
        user=config['swift_storage']['user']
        key=config['swift_storage']['key']
        auth_url=config['swift_storage']['auth_url']
        tenant_name=config['swift_storage']['tenant_name']
        auth_version=config['swift_storage']['auth_version']
        options = config['swift_storage']['options']
        self.conn = swiftclient.Connection(user=user,
                                      key=key,
                                      authurl=auth_url,
                                      os_options=options,
                                      tenant_name=tenant_name,
                                      auth_version=auth_version)
        return self.conn

    def upload(self, mask='c:\datasets\*.csv', prefix="/Trash/",content_type='text/csv', recursive=False):
        """
        Upload files to Open Stack Swift
        :param mask: str, Mask for seraching file to upload.
        :param prefix: str, Prefix in destination. Useful to mimic folders.
        :param content_type: str, Content type on the destination.
        :param recursive: boolean, To allow search in sub-folder.
        :return:
        """
        for file in tqdm(glob.glob(mask, recursive=recursive)):
            with open(file, 'rb') as one_file:
                    upload_to = prefix+ os.path.basename(file)
                    #print('Copy from',file,'to',upload_to)
                    self.conn.put_object(self.container_name, upload_to,
                                                    contents= one_file.read(),
                                                    content_type=content_type) # 'text/csv'
    def get_files_urls(self, pattern=""):
        result=[]
        objects = self.conn.get_container(self.container_name)[1]
        for data in objects:
            if pattern in data['name']:
                url = self.config['swift_storage']['base_url']+data['name']
                result.append(url)
        return result

    def delete_files(self, pattern="/Trash/", dry_run=True):
        if dry_run:
            print('Nothing will be deleted. Use dry_run=False to delete.')
        for data in self.conn.get_container(self.container_name)[1]:
            file = data['name']
            if pattern in file:
                print('deleting', file)
                if not dry_run:
                    self.conn.delete_object(self.container_name, file)
                   

    def get_containers(self):
        return self.conn.get_account()[1]
    def get_container(self, container_name='oco2'):
        return self.conn.get_container(container_name)[1]

    def get_url_from_sounding_id(self, sounding_id):
        return config['swift_storage']['base_url']+'/datasets/oco-2/peaks-detected-details/peak_data-si_'+sounding_id+'.json'
        
    def get_dataframe(self, url):
        """
        Read the url of a file and load it with Pandas
        :param url: str, URL of the file to load.
        :return: DataFrame
        """
        # TODO : Switch to GeoPandas ?
        df = None
        extension = url.split('.')[-1].lower()
        if extension == 'csv' or extension == 'xz':
            df = pd.read_csv(url, sep=';')
            if len(df.columns) == 1: # Very bad because we load it twice !
                df = pd.read_csv(url, sep=',')
            if 'sounding_id' in df.columns:
                df['sounding_id']= df['sounding_id'].astype(str)
        elif extension == 'json':
            df = pd.read_json(url)
        return df
    
    def get_gaussian_param(self, sounding_id, df_all_peak):
        df_param = df_all_peak.query("sounding_id==@sounding_id")
        if len(df_param)<1:
            print('ERROR : sounding_id not found in dataframe !')
            return {'slope' : 1,'intercept' : 1,'amplitude' : 1,'sigma': 1,'delta': 1,'R' : 1}
        param_index = df_param.index[0]

        gaussian_param = {
            'slope' : df_param.loc[param_index, 'slope'],
            'intercept' : df_param.loc[param_index, 'intercept'],
            'amplitude' : df_param.loc[param_index, 'amplitude'],
            'sigma': df_param.loc[param_index, 'sigma'],
            'delta': df_param.loc[param_index, 'delta'],
            'R' : df_param.loc[param_index, 'R'],
        }
        return gaussian_param

# Examples

## Connection

In [ ]:
config = './configs/config.json'
datasets = Datasets(config)

## Get a dataset

### Level 2 sounding value from OCO-2 satellite

In [ ]:
url=datasets.get_files_urls('/soudings/')[0]
print(url)
df=datasets.get_dataframe(url)
df.head(3)

https://storage.gra.cloud.ovh.net/v1/AUTH_2aaacef8e88a4ca897bb93b984bd04dd/oco2//datasets/oco-2/soudings/oco2_1409.csv.xz


,sounding_id,latitude,longitude,xco2,xco2_uncert,orbit,windspeed_u,windspeed_v,surface_pressure_apriori,surface_pressure,altitude,land_water_indicator,land_fraction
0,2014090602072531,54.465824,160.511734,394.609192,0.772341,958,-2.220395,0.993341,1013.956604,1012.595825,13.096467,0.0,100.0
1,2014090602072573,54.482216,160.537918,394.162170,0.677214,958,-2.180700,0.966224,1014.002808,1012.873962,12.102757,0.0,100.0
2,2014090602072574,54.480503,160.555984,395.652740,0.741387,958,-2.182690,0.930883,1014.672607,1014.506714,6.454920,0.0,100.0


### List of detected peak

In [ ]:
url=datasets.get_files_urls('peaks-detected/result')[0]
print(url)
df=datasets.get_dataframe(url)
df.head(3)

https://storage.gra.cloud.ovh.net/v1/AUTH_2aaacef8e88a4ca897bb93b984bd04dd/oco2//datasets/oco-2/peaks-detected/result_for_oco2_1409.csv


,Unnamed: 0,sounding_id,latitude,longitude,orbit,slope,intercept,amplitude,sigma,delta,R,windspeed_u,windspeed_v,surface_pressure_apriori,surface_pressure,altitude,land_water_indicator,land_fraction
0,0,2014090607005973,43.501789,90.922615,961,-0.007228,393.498364,29.462107,9.534737,1.232722,0.660609,2.995442,2.878621,777.691711,774.468628,2221.837158,0.0,100.0
1,1,2014090607010032,43.544865,90.895233,961,-0.007340,393.459130,31.360344,10.894470,1.148378,0.693845,2.847558,2.515943,774.478516,773.318420,2257.781982,0.0,100.0
2,2,2014090704095436,-16.547655,143.959457,974,-0.000913,395.653969,36.590754,10.926841,1.335939,0.642492,-3.830105,0.613206,994.341064,996.813232,178.946243,0.0,100.0


### Level 2 sounding value for a peak

In [ ]:
url=datasets.get_files_urls('/peaks-detected-details/')[0]
print(url)
df=datasets.get_dataframe(url)
df.head(3)

https://storage.gra.cloud.ovh.net/v1/AUTH_2aaacef8e88a4ca897bb93b984bd04dd/oco2//datasets/oco-2/peaks-detected-details/peak_data-si_2014090607005973.json


,orbit,sounding_id,latitude,longitude,xco2,xco2_uncert,windspeed_u,windspeed_v,surface_pressure_apriori,surface_pressure,altitude,land_water_indicator,land_fraction,latitude_orig,longitude_orig,distance,xco2_enhancement
0,961,2014090607004536,42.636265,91.254890,394.385742,0.608847,1.066924,-0.115263,959.215515,956.433167,413.602203,0,100,16.679594,98.151855,-99.885402,0.727142
1,961,2014090607004502,42.630756,91.208755,394.208282,0.484068,1.061136,-0.146890,959.631592,955.195984,409.457794,0,100,16.679594,98.151855,-99.505273,0.549683
2,961,2014090607004535,42.639961,91.241791,394.180023,0.581653,1.080121,-0.070387,960.536621,956.597473,401.312225,0,100,16.679594,98.151855,-99.213240,0.521423


## Get containers names

In [ ]:
for container in datasets.get_containers():
    print('Container name:', container['name'])

Container name: oco2


## List files

## Open Stack directories structure

We do not store the original OCO-2 files from NASA.

* /emissions/ contains all the potential source of emissions : factories, power plants, cities...
* /soudings/ contains CSV of the raw features extracted from NASA NC4 files.
* /peaks-detected/ contains all the peak found in the satellite orbit datas.
* /peaks-detected-details/ contains one JSON file of the full data for all detected peak

In [ ]:
datasets.get_files_urls('html')

['https://storage.gra.cloud.ovh.net/v1/AUTH_2aaacef8e88a4ca897bb93b984bd04dd/oco2//map/peaks_and_sources.html']

### Get files objects

In [ ]:
objects = datasets.get_container('oco2')
for data in objects:
    if 'oco2_1504' in data['name']:
        print('{0}\t{1}\t{2}'.format(data['name'], data['bytes'], data['last_modified']))


/datasets/oco-2/peaks-detected/result_for_oco2_1504.csv	21241	2020-05-05T17:13:30.884370
/datasets/oco-2/soudings/oco2_1504.csv.xz	75186100	2020-05-03T07:48:47.793680


## Upload files

In [ ]:
datasets.upload(mask='../*.md', prefix="/Trash/",content_type='text/text')
# datasets.("/media/data-nvme/dev/datasets/OCO2/csv/*.csv", "/datasets/oco-2/peaks-detected/", 'text/csv')
# datasets.("/media/data-nvme/dev/datasets/OCO2/csv/*.json", "/datasets/oco-2/peaks-detected-details/", 'application/json')

100%|██████████| 3/3 [00:02<00:00,  1.15it/s]


In [ ]:
datasets.get_files_urls('/Trash/')

['https://storage.gra.cloud.ovh.net/v1/AUTH_2aaacef8e88a4ca897bb93b984bd04dd/oco2//Trash/CONTRIBUTING.md',
 'https://storage.gra.cloud.ovh.net/v1/AUTH_2aaacef8e88a4ca897bb93b984bd04dd/oco2//Trash/README-old.md',
 'https://storage.gra.cloud.ovh.net/v1/AUTH_2aaacef8e88a4ca897bb93b984bd04dd/oco2//Trash/README.md']

### Upload HTML
Setting content type to 'text/html' allow the file to be display by browsers, without downloading.

In [ ]:
#datasets.upload("chemin/peaks_and_sources.html", "/Trash/", 'text/html')

## Delete files

In [ ]:
datasets.delete_files("/Trash/", dry_run=False)

deleting /Trash/CONTRIBUTING.md
deleting /Trash/README-old.md
deleting /Trash/README.md


In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 03_25_OCO2_Data_Exploration.ipynb.
Converted 04_01_OCO2_Work_Base.ipynb.
Converted 04_04_OCO2_China_Peaks.ipynb.
Converted 04_15_OCO2_Laiwu_Peak_Detection.ipynb.
Converted CO2_emissions_Inventory_data.ipynb.
Converted Christian-datasets-Distances.ipynb.
Converted Find_Peaks_with_LSTM_autoencoders.ipynb.
Converted Laiwu_Plume-more_data.ipynb.
Converted Laiwu_Plume-more_data_CD_exploration_selection_peaks.ipynb.
Converted Laiwu_Plume.ipynb.
Converted Untitled.ipynb.
Converted WIP_OCO2_Capture.ipynb.
Converted WIP_OCO2_Peaks_Wind.ipynb.
Converted WIP_OCO2_Peaks_Wind_Visualization.ipynb.
Converted find_peak_bco_test.ipynb.
Converted index.ipynb.
Converted oco2peak-datasets.ipynb.
Converted oco2peak-find_peak.ipynb.
Converted oco2peak-map.ipynb.
Converted oco2peak-nc4_convert.ipynb.
Converted oco2peak-swift_utils.ipynb.
Converted show_map.ipynb.
Converted view_peak.ipynb.
